# r5 Data Preparation

* Uses methods from "./r5_data_preparation.py" to prep data for the computation of Travel-Time Matrices with r5.
  
* That includes:
    * Download **OSM-Network Data** (.pbf) and cut it via BoundingBox (shapely.polygon e.g. WKT)
        * https://download.geofabrik.de
    * Export gtfs Geometries to gpkg

* For **filtering GTFS-Data** use the R-Skript, experiments with Python module "gtfs-kit" were not successful and caused errors with r5-engine

In [1]:
import os
import geopandas as gpd

# del sys.modules["r5_data_preparation"]
from r5_data_preparation import (
    OSM_PBF_zuschneiden,
    buffer_geometries,
    GTFS_to_gpkg
)

In [2]:
# pfade aus config holen
# init configParser and open file
repoDirectoryPath = ''

Kreise_MRH_Zensus = os.path.join(repoDirectoryPath, '')
gdf_kreise = gpd.GeoDataFrame.from_file(Kreise_MRH_Zensus)

## OSM Network Data laden und auf Kreise zuschneiden

In [ ]:
# falls noch nicht geschehen kann so die osm-pbf--Dateien für ein Bundesland geladen werden
fp = pyrosm.get_data(bundesland, update=False,directory=download_directory)


In [ ]:
out_dir = os.path.join(repoDirectoryPath, 'Daten/Output/Netzwerk/zugeschnitten/MRH')

for index, row in gdf_kreise.iterrows():

    kreis = gpd.GeoDataFrame([row], columns=gdf_kreise.columns, crs=gdf_kreise.crs)
    kreis_buffered = buffer_geometries(kreis, 20)   # buffer von 20km damit auch mittellange strecken aus dem kreis gefahren werden können
    
    if kreis['AGS'].iloc[0][:2] == '01':    #SH
        in_pbf = os.path.join(repoDirectoryPath, '/Netzwerk/osm/schleswig-holstein-latest.osm.pbf')
    elif kreis['AGS'].iloc[0][:2] == '02':  # Ham
        in_pbf = os.path.join(repoDirectoryPath, '/Netzwerk/osm/Hamburg.osm.pbf')
    elif kreis['AGS'].iloc[0][:2] == '03':  # Niedersachsen
        in_pbf = os.path.join(repoDirectoryPath, '/Netzwerk/osm/niedersachsen-latest.osm.pbf')
    elif kreis['AGS'].iloc[0][:2] == '13':  # MckPomm
        in_pbf = os.path.join(repoDirectoryPath, '/Netzwerk/osm/mecklenburg-vorpommern-latest.osm.pbf')

    result = OSM_PBF_zuschneiden(name_kreis = kreis['GEN'].iloc[0], 
                        bounding_gdf = kreis_buffered, 
                        in_pbf = in_pbf, 
                        out_dir = out_dir, 
                        export_to_gpkg=True)
    
    if result.returncode == 0:
        print(f'Zugeschnittenes pbf-File unter {result.args[8]} abgelegt')
    else:
        print(f"Fehler bei Osmium extract: {result.stderr}")


## Export GTFS to gpkg

In [6]:
types = ['routes','shapes','trips','stops']
for index, row in gdf_kreise.iterrows():
    name_kreis = row['GEN']
    in_gtfs_zip = os.path.join(repoDirectoryPath, f'Daten/Output/GTFS/zugeschnitten/MRH/DELFI/DELFI_gtfs_filtered_{name_kreis}.zip')
    out_dir = os.path.join(repoDirectoryPath, f'Daten/Output/GTFS/gpkg/DELFI_gtfs_filtered_{name_kreis}')
    print(f'GPKGs gespeichert: {name_kreis}')
    GTFS_to_gpkg(in_gtfs_zip, out_dir, types)

GPKGs gespeichert: Lübeck
GPKGs gespeichert: Neumünster
GPKGs gespeichert: Dithmarschen
GPKGs gespeichert: Herzogtum Lauenburg
GPKGs gespeichert: Ostholstein
GPKGs gespeichert: Pinneberg
GPKGs gespeichert: Segeberg
GPKGs gespeichert: Steinburg
GPKGs gespeichert: Stormarn
GPKGs gespeichert: Hamburg
GPKGs gespeichert: Cuxhaven
GPKGs gespeichert: Harburg
GPKGs gespeichert: Lüchow-Dannenberg
GPKGs gespeichert: Lüneburg
GPKGs gespeichert: Rotenburg (Wümme)
GPKGs gespeichert: Heidekreis
GPKGs gespeichert: Stade
GPKGs gespeichert: Uelzen
GPKGs gespeichert: Schwerin
GPKGs gespeichert: Nordwestmecklenburg
GPKGs gespeichert: Ludwigslust-Parchim
